<a href="https://colab.research.google.com/github/Ladvika/Trading-Strategy-Agent-/blob/main/Trading_Strategy_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# trading_agent_app.py
# FULL STACK (Frontend + Backend + Database) in ONE FILE

from flask import Flask, request, jsonify
import sqlite3, time, random, math

app = Flask(__name__)

DB = "trading_agent.db"

# -------------------------
# DATABASE SETUP
# -------------------------
def init_db():
    con = sqlite3.connect(DB)
    cur = con.cursor()

    # store compressed embeddings/patterns
    cur.execute("""
        CREATE TABLE IF NOT EXISTS market_patterns (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp INTEGER,
            symbol TEXT,
            regime TEXT,
            embedding TEXT,
            outcome REAL,
            confidence REAL
        )
    """)

    # store trades
    cur.execute("""
        CREATE TABLE IF NOT EXISTS trades (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp INTEGER,
            symbol TEXT,
            action TEXT,
            price REAL,
            stop_loss REAL,
            take_profit REAL,
            confidence REAL
        )
    """)
    con.commit()
    con.close()

init_db()

# -------------------------
# MARKET SIMULATION (FAKE DATA)
# (You can replace this with real Binance/CCXT later)
# -------------------------
def get_market_data(symbol="BTCUSDT"):
    price = round(65000 + random.uniform(-500, 500), 2)
    volume = round(random.uniform(100, 1000), 2)
    spread = round(random.uniform(0.5, 3.0), 2)
    return price, volume, spread

# -------------------------
# FEATURE ENGINE
# -------------------------
def compute_features(price, volume, spread):
    # dummy features
    rsi = round(random.uniform(20, 80), 2)
    atr = round(random.uniform(50, 300), 2)
    momentum = round(random.uniform(-1, 1), 4)
    vol_spike = 1 if volume > 700 else 0
    return {"rsi": rsi, "atr": atr, "momentum": momentum, "spread": spread, "vol_spike": vol_spike}

# -------------------------
# MARKET COMPRESSION (Embedding)
# -------------------------
def compress_market(features):
    # simple compression into 8-dim "embedding"
    emb = [
        features["rsi"] / 100,
        features["atr"] / 500,
        (features["momentum"] + 1) / 2,
        features["spread"] / 10,
        features["vol_spike"],
        math.sin(features["rsi"]),
        math.cos(features["rsi"]),
        (features["atr"] % 100) / 100
    ]
    # store as string for SQLite
    return ",".join([str(round(x, 6)) for x in emb])

# -------------------------
# REGIME CLASSIFIER
# -------------------------
def classify_regime(features):
    if features["atr"] > 200 and features["vol_spike"] == 1:
        return "R2_BREAKOUT_HIGH_VOL"
    elif features["momentum"] > 0.4:
        return "R4_STRONG_UPTREND"
    elif features["momentum"] < -0.4:
        return "R3_STRONG_DOWNTREND"
    else:
        return "R1_SIDEWAYS"

# -------------------------
# DECISION ENGINE
# -------------------------
def generate_signal(regime, features):
    confidence = round(random.uniform(0.55, 0.9), 2)

    if regime == "R4_STRONG_UPTREND" and features["rsi"] < 75:
        action = "BUY"
    elif regime == "R3_STRONG_DOWNTREND" and features["rsi"] > 25:
        action = "SELL"
    elif regime == "R2_BREAKOUT_HIGH_VOL":
        action = "BUY" if features["momentum"] > 0 else "SELL"
    else:
        action = "HOLD"
        confidence = round(random.uniform(0.4, 0.6), 2)

    return action, confidence

# -------------------------
# RISK MANAGER
# -------------------------
def risk_levels(price, features, action):
    atr = features["atr"]

    if action == "BUY":
        stop_loss = price - atr
        take_profit = price + (2 * atr)
    elif action == "SELL":
        stop_loss = price + atr
        take_profit = price - (2 * atr)
    else:
        stop_loss = None
        take_profit = None

    return stop_loss, take_profit

# -------------------------
# DATABASE HELPERS
# -------------------------
def save_pattern(symbol, regime, embedding, outcome, confidence):
    con = sqlite3.connect(DB)
    cur = con.cursor()
    cur.execute("""
        INSERT INTO market_patterns(timestamp, symbol, regime, embedding, outcome, confidence)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (int(time.time()), symbol, regime, embedding, outcome, confidence))
    con.commit()
    con.close()

def save_trade(symbol, action, price, stop_loss, take_profit, confidence):
    con = sqlite3.connect(DB)
    cur = con.cursor()
    cur.execute("""
        INSERT INTO trades(timestamp, symbol, action, price, stop_loss, take_profit, confidence)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (int(time.time()), symbol, action, price, stop_loss, take_profit, confidence))
    con.commit()
    con.close()

# -------------------------
# API ROUTES
# -------------------------
@app.route("/api/decision", methods=["GET"])
def api_decision():
    symbol = request.args.get("symbol", "BTCUSDT")

    price, volume, spread = get_market_data(symbol)
    features = compute_features(price, volume, spread)
    embedding = compress_market(features)
    regime = classify_regime(features)
    action, confidence = generate_signal(regime, features)
    stop_loss, take_profit = risk_levels(price, features, action)

    # Save compressed pattern always (learning memory)
    save_pattern(symbol, regime, embedding, outcome=random.uniform(-2, 2), confidence=confidence)

    # Save trade only if action is not HOLD
    if action != "HOLD":
        save_trade(symbol, action, price, stop_loss, take_profit, confidence)

    return jsonify({
        "symbol": symbol,
        "price": price,
        "volume": volume,
        "spread": spread,
        "features": features,
        "embedding": embedding,
        "regime": regime,
        "signal": action,
        "confidence": confidence,
        "stop_loss": stop_loss,
        "take_profit": take_profit
    })

@app.route("/api/trades", methods=["GET"])
def api_trades():
    con = sqlite3.connect(DB)
    cur = con.cursor()
    cur.execute("SELECT id, timestamp, symbol, action, price, stop_loss, take_profit, confidence FROM trades ORDER BY id DESC LIMIT 20")
    rows = cur.fetchall()
    con.close()

    trades = []
    for r in rows:
        trades.append({
            "id": r[0],
            "timestamp": r[1],
            "symbol": r[2],
            "action": r[3],
            "price": r[4],
            "stop_loss": r[5],
            "take_profit": r[6],
            "confidence": r[7]
        })
    return jsonify(trades)

# -------------------------
# FRONTEND (IN SAME FILE)
# -------------------------
@app.route("/")
def index():
    return """
<!DOCTYPE html>
<html>
<head>
    <title>Trading Strategy Agent Dashboard</title>
    <style>
        body{font-family:Arial;background:#0b0f1a;color:white;padding:20px;}
        h1{color:#00ffaa;}
        .card{background:#111a2e;padding:15px;border-radius:12px;margin:12px 0;box-shadow:0 0 8px rgba(0,255,170,0.2);}
        button{background:#00ffaa;color:black;font-weight:bold;padding:10px 15px;border:none;border-radius:10px;cursor:pointer;}
        button:hover{opacity:0.9;}
        pre{white-space:pre-wrap;word-wrap:break-word;}
        .buy{color:#00ff55;font-weight:bold;}
        .sell{color:#ff4444;font-weight:bold;}
        .hold{color:#ffaa00;font-weight:bold;}
        table{width:100%;border-collapse:collapse;}
        th,td{padding:8px;border-bottom:1px solid #223;}
    </style>
</head>
<body>

<h1>📈 Trading Strategy Agent</h1>

<div class="card">
    <h2>Live Decision</h2>
    <button onclick="getDecision()">Get Signal</button>
    <p id="signalText"></p>
    <pre id="decisionBox">Click "Get Signal"</pre>
</div>

<div class="card">
    <h2>Recent Trades (Last 20)</h2>
    <button onclick="loadTrades()">Refresh Trades</button>
    <div id="tradeBox">No trades loaded.</div>
</div>

<script>
async function getDecision(){
    const res = await fetch("/api/decision?symbol=BTCUSDT");
    const data = await res.json();

    let cls = "hold";
    if(data.signal==="BUY") cls="buy";
    if(data.signal==="SELL") cls="sell";

    document.getElementById("signalText").innerHTML =
        "Signal: <span class='"+cls+"'>" + data.signal + "</span> | Confidence: " + data.confidence;

    document.getElementById("decisionBox").textContent = JSON.stringify(data, null, 2);

    loadTrades(); // auto update
}

async function loadTrades(){
    const res = await fetch("/api/trades");
    const trades = await res.json();

    if(trades.length === 0){
        document.getElementById("tradeBox").innerHTML = "No trades found.";
        return;
    }

    let html = "<table><tr><th>ID</th><th>Symbol</th><th>Action</th><th>Price</th><th>SL</th><th>TP</th><th>Conf</th></tr>";
    trades.forEach(t=>{
        html += `<tr>
            <td>${t.id}</td>
            <td>${t.symbol}</td>
            <td>${t.action}</td>
            <td>${t.price}</td>
            <td>${t.stop_loss ?? "-"}</td>
            <td>${t.take_profit ?? "-"}</td>
            <td>${t.confidence}</td>
        </tr>`;
    });
    html += "</table>";
    document.getElementById("tradeBox").innerHTML = html;
}
</script>

</body>
</html>
"""

# -------------------------
# OUTPUT / RUN SERVER
# -------------------------
if __name__ == "__main__":
    print("✅ Trading Strategy Agent Running!")
    print("🌐 Open in browser: http://127.0.0.1:5000")
    app.run(debug=True)


✅ Trading Strategy Agent Running!
🌐 Open in browser: http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
